In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# p_f = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\20250221_0g-ov-27.5d_6'


In [3]:
# 对齐时间
def get_laser_on_and_off(s_data):
    # 提取每个激光照射起始和终止时刻
    s_start = s_data[s_data[1] == 2].index
    s_stop = s_data[s_data[1] == 3].index
    if s_stop.empty:
        s_s = [len(s_data[1])]
    else:
        s_s = []
        for m in s_start:
            # 如果有多个照射片段，分别找到结束的时间装入s_s
            data_1 = (s_stop >= m)
            s_s.append(s_stop[data_1 == 1][0])

    i=0  # 不考虑其它段，只考虑第一段
    #         t_l = int(round((s_data[0][s_start[i] + 1 : s_s[i] - 1]).diff().sum(skipna=True) / (s_s[i] - s_start[i] - 2)))
    t_l = (s_data[0][s_start[i] + 1 : s_s[i] - 1]).diff().sum(skipna=True) / (s_s[i] - s_start[i] - 2)
    print('时间间隔t_l',t_l)
    t = s_data[0].values
    # 取从激光照射开始前2帧到激光照射结束的数据
    t_frame = np.concatenate([[t[s_start[i]] - t_l *2, t[s_start[i]] - t_l], t[s_start[i]:s_s[i]]])
    t_start = s_data[0][s_start[i]] - t_l *2
    t_stop = s_data[0][s_s[i] - 1] + t_l
    return t_frame, t_start, t_stop
def get_nearest_value(df1, df2, col_1 = 'Time', col_2 = 'Vol_Time', ind_col = 'Nearest_Index', col1_copy = 'Nearest_Time'):
    '''
    根据df2的col2找最接近的df1中的col1的值生成新列赋值与df2
    '''
    nearest_times = []
    nearest_indices = []
    for vol_time in df2[col_2]:
        # 计算时间差的绝对值
        differences = np.abs(df1[col_1] - vol_time)
        # 找到最小差值的索引
        nearest_idx = np.argmin(differences)
        # 保存最近时间值和索引
        nearest_times.append(df1.iloc[nearest_idx, df1.columns.get_loc(col_1)])
        nearest_indices.append(nearest_idx)

    # 将结果存入 DataFrame
    df2[col1_copy] = nearest_times
    df2[ind_col] = nearest_indices

    return df2
def get_df_mot_matched_vol_t(df_mat_cut,p_f, df_vol, ext_name = '', write=True):
    '''
    荧光对时: 根据df_vol_time中'Time'提取df_mat_cut中的数据列并写出
    '''
    folder_name = os.path.basename(os.path.normpath(p_f))
    df_vol = get_nearest_value(df_mat_cut, df_vol, col_1 = 'Time', col_2 = 'Vol_Time', ind_col = 'Nearest_Index', col1_copy = 'Nearest_Time')
    # 将df_motionz中的运动参数(有中线加上中线)按照索引对应到视频帧文件的时间戳上
    columns_to_add = df_mat_cut.columns
    for col in columns_to_add:
        df_vol[col] = df_mat_cut.iloc[df_vol['Nearest_Index'].values][col].values
        fn_motion_output = folder_name+ext_name
    if write:
        end = ext_name.split('.')[-1]
        if end == 'pkl':
            df_vol.to_pickle(os.path.join(p_f, fn_motion_output))
        elif end =='csv':
            df_vol.to_csv(os.path.join(p_f, fn_motion_output), index = False)
    print(f'写出与神经数据时间戳对应的运动/中线参数文件到:{fn_motion_output}')
    return df_vol

In [4]:
p_f_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\nomask'
nokey_word = '*'
key_word = ''
files = [f_p for f_p in os.listdir(p_f_all) if ('trash' not in f_p)&
              ('incomplete' not in f_p)&
             os.path.isdir(os.path.join(p_f_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]
print('\n'.join(files))

20241219_0g-ov_05
20250115_4.5g-24d-ov_08
20250116_4.5g-ov_05


In [5]:
def auto_nomask_all(p_f):  
    # 求t_frame
    files = [f for f in os.listdir(p_f)]
    folder_name = os.path.basename(os.path.normpath(p_f))
    for f in files:
        if ('laser-data' in f)or ("s-data" in f):
            # 读取激光激活数据
            print('读取激光照射时间数据并转为s_data')
            try:
                s_data = pd.read_csv(p_f + '\laser-data.txt', header=None)  #读取激光激活数据
            except FileNotFoundError:
                print('读取')
                s_data = pd.read_csv(p_f + '\s-data.txt', header=None)
            # 注意：这里s_data已经转换成秒
            s_data.loc[:,0] = s_data.loc[:,0]/1000
        elif 'calcium_intensity.npy' in f:
                    # 钙信号数据(* 如果有)
                    print(f'读取钙信号数据：{p_f+f}')
                    Cal_data_path = os.path.join(p_f, f)
                    calcium_intensity= np.load(Cal_data_path)

    # 步骤五：根据激光开始和结束时间截取荧光拍摄部分的运动数据/时间戳
    t_frame, t_start, t_stop = get_laser_on_and_off(s_data)
    # 读入df_mot_midline_cut
    files = [f for f in os.listdir(p_f) if '_mot_midline_cut' in f]
    # 优先找 .pkl
    pkl_files = [f for f in files if f.endswith('.pkl')]
    csv_files = [f for f in files if f.endswith('.csv')]

    if len(pkl_files):
        f_mot_mid_cut = pkl_files[0]   # 如果有多个 .pkl，只取第一个
        df_mot_midline_cut = pd.read_pickle(os.path.join(p_f,f_mot_mid_cut))
    elif len(csv_files):
        f_mot_mid_cut = csv_files[0]   # 没有 .pkl，但有 .csv
        df_mot_midline_cut = pd.read_csv(os.path.join(p_f,f_mot_mid_cut))
    else:
        raise FileNotFoundError("没有找到符合条件的 _mot_midline_cut 文件")
    print(f_mot_mid_cut)


    # 如果没有mask
    df_vol_time = pd.DataFrame(t_frame, columns=['Vol_Time'])   # 激光照射时间戳要转换单位为s
    df_vol_time.loc[:,'mask'] = 0
    df_vol_time.to_csv(os.path.join(p_f, folder_name+'_VotTime.csv'), index = False)
    # 有中线, 如果需要mask，write = False
    df_mot_mid_vol = get_df_mot_matched_vol_t(df_mot_midline_cut,p_f, df_vol_time.copy(),
                                            ext_name = '_MotionMidlineMatchVol.pkl', write=True)

In [6]:
for f in files:
    auto_nomask_all(os.path.join(p_f_all,f))

读取钙信号数据：Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\nomask\20241219_0g-ov_05calcium_intensity.npy
读取激光照射时间数据并转为s_data
读取
时间间隔t_l 0.29993044783587686
20241219_0g-ov_05_mot_midline_cut.csv
写出与神经数据时间戳对应的运动/中线参数文件到:20241219_0g-ov_05_MotionMidlineMatchVol.pkl
读取钙信号数据：Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\nomask\20250115_4.5g-24d-ov_08calcium_intensity.npy
读取激光照射时间数据并转为s_data
读取
时间间隔t_l 0.29990721986278995
20250115_4.5g-24d-ov_08_mot_midline_cut.csv
写出与神经数据时间戳对应的运动/中线参数文件到:20250115_4.5g-24d-ov_08_MotionMidlineMatchVol.pkl
读取钙信号数据：Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done\\nomask\20250116_4.5g-ov_05calcium_intensity.npy
读取激光照射时间数据并转为s_data
读取
时间间隔t_l 0.29992736842105266
20250116_4.5g-ov_05_mot_midline_cut.csv
写出与神经数据时间戳对应的运动/中线参数文件到:20250116_4.5g-ov_05_MotionMidlineMatchVol.pkl
